ВЫПУСКНОЙ ПРОЕКТ

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import os
from scipy import stats as st
import math as mth
from plotly import graph_objects as go

## Чтение и предобработка

In [2]:
mobile_sources = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_soures.csv')
mobile_dataset = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_dataset.csv')

In [3]:
display(mobile_sources)
mobile_dataset

,userId,source
0,020292ab-89bc-4156-9acf-68bc2783f894,other
1,cf7eda61-9349-469f-ac27-e5b6f5ec475c,yandex
2,8c356c42-3ba9-4cb6-80b8-3f868d0192c3,yandex
3,d9b06b47-0f36-419b-bbb0-3533e582a6cb,other
4,f32e1e2a-3027-4693-b793-b7b3ff274439,google
...,...,...
4288,b86fe56e-f2de-4f8a-b192-cd89a37ecd41,yandex
4289,424c0ae1-3ea3-4f1e-a814-6bac73e48ab1,yandex
4290,437a4cd4-9ba9-457f-8614-d142bc48fbeb,yandex
4291,c10055f0-0b47-477a-869e-d391b31fdf8f,yandex


,event.time,event.name,user.id
0,2019-10-07 00:00:00.431357,advert_open,020292ab-89bc-4156-9acf-68bc2783f894
1,2019-10-07 00:00:01.236320,tips_show,020292ab-89bc-4156-9acf-68bc2783f894
2,2019-10-07 00:00:02.245341,tips_show,cf7eda61-9349-469f-ac27-e5b6f5ec475c
3,2019-10-07 00:00:07.039334,tips_show,020292ab-89bc-4156-9acf-68bc2783f894
4,2019-10-07 00:00:56.319813,advert_open,cf7eda61-9349-469f-ac27-e5b6f5ec475c
...,...,...,...
74192,2019-11-03 23:53:29.534986,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9
74193,2019-11-03 23:54:00.407086,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9
74194,2019-11-03 23:56:57.041825,search_1,20850c8f-4135-4059-b13b-198d3ac59902
74195,2019-11-03 23:57:06.232189,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9


In [4]:
mobile_sources = mobile_sources.rename(columns={'userId':'user_id'})
mobile_dataset = mobile_dataset.rename(columns={'event.time':'event_dt', 'event.name':'event_name', 'user.id':'user_id'})
display(mobile_sources.info())
mobile_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4293 entries, 0 to 4292
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  4293 non-null   object
 1   source   4293 non-null   object
dtypes: object(2)
memory usage: 67.2+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74197 entries, 0 to 74196
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   event_dt    74197 non-null  object
 1   event_name  74197 non-null  object
 2   user_id     74197 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB


In [5]:
mobile_dataset['event_dt'] = pd.to_datetime(mobile_dataset['event_dt'], format='%Y-%m-%d %H:%M:%S')
mobile_dataset

,event_dt,event_name,user_id
0,2019-10-07 00:00:00.431357,advert_open,020292ab-89bc-4156-9acf-68bc2783f894
1,2019-10-07 00:00:01.236320,tips_show,020292ab-89bc-4156-9acf-68bc2783f894
2,2019-10-07 00:00:02.245341,tips_show,cf7eda61-9349-469f-ac27-e5b6f5ec475c
3,2019-10-07 00:00:07.039334,tips_show,020292ab-89bc-4156-9acf-68bc2783f894
4,2019-10-07 00:00:56.319813,advert_open,cf7eda61-9349-469f-ac27-e5b6f5ec475c
...,...,...,...
74192,2019-11-03 23:53:29.534986,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9
74193,2019-11-03 23:54:00.407086,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9
74194,2019-11-03 23:56:57.041825,search_1,20850c8f-4135-4059-b13b-198d3ac59902
74195,2019-11-03 23:57:06.232189,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9


In [6]:
mobile_dataset = mobile_dataset.sort_values(['user_id', 'event_dt']) 
mobile_dataset['session_dt'] = pd.DatetimeIndex(mobile_dataset['event_dt']).date
mobile_dataset['session_dt'] = pd.to_datetime(mobile_dataset['session_dt'])
mobile_dataset.info()
mobile_dataset

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74197 entries, 805 to 72689
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   event_dt    74197 non-null  datetime64[ns]
 1   event_name  74197 non-null  object        
 2   user_id     74197 non-null  object        
 3   session_dt  74197 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.8+ MB


,event_dt,event_name,user_id,session_dt
805,2019-10-07 13:39:45.989359,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-07
806,2019-10-07 13:40:31.052909,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-07
809,2019-10-07 13:41:05.722489,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-07
820,2019-10-07 13:43:20.735461,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-07
830,2019-10-07 13:45:30.917502,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-07
...,...,...,...,...
72584,2019-11-03 15:51:23.959572,tips_show,fffb9e79-b927-4dbb-9b48-7fd09b23a62b,2019-11-03
72589,2019-11-03 15:51:57.899997,contacts_show,fffb9e79-b927-4dbb-9b48-7fd09b23a62b,2019-11-03
72684,2019-11-03 16:07:40.932077,tips_show,fffb9e79-b927-4dbb-9b48-7fd09b23a62b,2019-11-03
72688,2019-11-03 16:08:18.202734,tips_show,fffb9e79-b927-4dbb-9b48-7fd09b23a62b,2019-11-03


In [7]:
display(mobile_dataset.isna().sum())
mobile_sources.isna().sum()

# magic - никаких пропусков!

event_dt      0
event_name    0
user_id       0
session_dt    0
dtype: int64

user_id    0
source     0
dtype: int64

In [8]:
display(mobile_dataset.duplicated().sum())
mobile_sources.duplicated().sum()

# и снова magic - никаких дупликатов! мне не кажется, что тут имеет смысл проверять на неявные дупликаты
# слишком много разных параметров. 

0

0

In [9]:
display(mobile_dataset['event_name'].unique())
mobile_sources['source'].unique()

array(['tips_show', 'map', 'search_1', 'photos_show', 'favorites_add',
       'contacts_show', 'contacts_call', 'advert_open', 'search_7',
       'search_5', 'search_4', 'search_6', 'search_3', 'tips_click',
       'search_2', 'show_contacts'], dtype=object)

array(['other', 'yandex', 'google'], dtype=object)

In [10]:
# переименовать события внутри (серчи в один точно), целевое событие привести к 1 виду

mobile_dataset = mobile_dataset.replace({'event_name':{'search_1':'search', 'search_2':'search','search_3':'search',
                                                       'search_4':'search','search_5':'search','search_6':'search',
                                                      'search_7':'search', 'show_contacts':'contacts_show'}})
mobile_dataset['event_name'].unique()

array(['tips_show', 'map', 'search', 'photos_show', 'favorites_add',
       'contacts_show', 'contacts_call', 'advert_open', 'tips_click'],
      dtype=object)

In [11]:
mobile_dataset.duplicated().sum()

# даже после этого дупликатов не находится

0

In [12]:
# объединить два датафрейма

mobile_df = mobile_dataset.merge(mobile_sources, on='user_id')

mobile_df.info()
display(mobile_df)
mobile_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74197 entries, 0 to 74196
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   event_dt    74197 non-null  datetime64[ns]
 1   event_name  74197 non-null  object        
 2   user_id     74197 non-null  object        
 3   session_dt  74197 non-null  datetime64[ns]
 4   source      74197 non-null  object        
dtypes: datetime64[ns](2), object(3)
memory usage: 3.4+ MB


,event_dt,event_name,user_id,session_dt,source
0,2019-10-07 13:39:45.989359,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-07,other
1,2019-10-07 13:40:31.052909,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-07,other
2,2019-10-07 13:41:05.722489,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-07,other
3,2019-10-07 13:43:20.735461,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-07,other
4,2019-10-07 13:45:30.917502,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-07,other
...,...,...,...,...,...
74192,2019-11-03 15:51:23.959572,tips_show,fffb9e79-b927-4dbb-9b48-7fd09b23a62b,2019-11-03,google
74193,2019-11-03 15:51:57.899997,contacts_show,fffb9e79-b927-4dbb-9b48-7fd09b23a62b,2019-11-03,google
74194,2019-11-03 16:07:40.932077,tips_show,fffb9e79-b927-4dbb-9b48-7fd09b23a62b,2019-11-03,google
74195,2019-11-03 16:08:18.202734,tips_show,fffb9e79-b927-4dbb-9b48-7fd09b23a62b,2019-11-03,google


event_dt      0
event_name    0
user_id       0
session_dt    0
source        0
dtype: int64

In [13]:
display(len(mobile_df['user_id'].unique()))
display(len(mobile_dataset['user_id'].unique()))
len(mobile_sources['user_id'].unique())

# этл чек для себя, что все уникальные пользователи сохранились и в процессе предобработки ничего не потерялось

4293

4293

4293

## Расчет retention rate

### Создаю профили пользователей

In [14]:
def get_profiles(sessions):

    # сортируем сессии по ID пользователя и дате первого посещения
    # группируем по ID и находим первые значения session_start и channel
    # столбец с временем первого посещения назовём first_ts
    # от англ. first timestamp — первая временная отметка
    profiles = (
        mobile_df.sort_values(by=['user_id', 'session_dt'])
        .groupby('user_id')
        .agg({'session_dt': 'first', 'event_name':'first', 'source': 'first'})
        .rename(columns={'session_dt': 'first_ts'})
        .reset_index()  # возвращаем user_id из индекса
    )

    # определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    # эти данные понадобятся для когортного анализа
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    return profiles 

нужно создать столбец first_ts для того, чтобы чтобы посчитать лайфтаймы у всех. 
это будет мин дата для каждого уникального юзера

In [15]:
profiles = get_profiles(mobile_df)
profiles

,user_id,first_ts,event_name,source,dt,month
0,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-07,tips_show,other,2019-10-07,2019-10-01
1,00157779-810c-4498-9e05-a1e9e3cedf93,2019-10-19,search,yandex,2019-10-19,2019-10-01
2,00463033-5717-4bf1-91b4-09183923b9df,2019-11-01,photos_show,yandex,2019-11-01,2019-11-01
3,004690c3-5a84-4bb7-a8af-e0c8f8fca64e,2019-10-18,search,google,2019-10-18,2019-10-01
4,00551e79-152e-4441-9cf7-565d7eb04090,2019-10-25,contacts_show,yandex,2019-10-25,2019-10-01
...,...,...,...,...,...,...
4288,ffab8d8a-30bb-424a-a3ab-0b63ebbf7b07,2019-10-13,map,yandex,2019-10-13,2019-10-01
4289,ffc01466-fdb1-4460-ae94-e800f52eb136,2019-10-07,photos_show,yandex,2019-10-07,2019-10-01
4290,ffcf50d9-293c-4254-8243-4890b030b238,2019-10-23,tips_show,google,2019-10-23,2019-10-01
4291,ffe68f10-e48e-470e-be9b-eeb93128ff1a,2019-10-21,search,yandex,2019-10-21,2019-10-01


## время, проведённое в приложении

##  частота действий

## конверсия в целевое действие — просмотр контактов.

## Сегментация пользователей

## Проверка стат. гипотез